In [18]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [19]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [28]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.RNN as rnn

class GRU(rnn.RNN):

    def __init__(self, D, H, char2idx, idx2char):
        super().__init__(D, H, char2idx, idx2char)

    def forward(self, X, h_old, train=True):
        m = self.model
        Wz, Wh, Wy = m['Wz'], m['Wh'], m['Wy']
        bz, bh, by = m['bz'], m['bh'], m['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        # concat: [h, x]
        X = np.column_stack((h_old, X_one_hot))

        # gate: h_prob
        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        # signal: h_pred
        hh, hh_cache = l.fc_forward(X, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        # h_next and y_pred
        #         h = (1. - hz) * h_old + hz * hh
        h = h_old + hz * (hh - h_old)
        y, y_cache = l.fc_forward(h, Wy, by)

        cache = h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, y_pred, y_train, dh_next, cache):
        h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache

        dy = loss_fun.dcross_entropy(y_pred, y_train)

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dh_old1 = (1. - hz) * dh

        dhh = hz * dh
        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX, dWh, dbh = l.fc_backward(dhh, hh_cache)
        dh_old2 = dX[:, :self.H]

        #         dhz = hh * dh - h_old * dh
        dhz = (hh - h_old) * dh
        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dX, dWz, dbz = l.fc_backward(dhz, hz_cache)
        dh_old3 = dX[:, :self.H]

        dh_next = dh_old1 + dh_old2 + dh_old3

        grad = dict(Wz=dWz, Wh=dWh, Wy=dWy, bz=dbz, bh=dbh, by=dby)

        return grad, dh_next

    def _init_model(self, D, C, H):
        Z = H + D

        self.model = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(D / 2.),
            bz=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [29]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 10
n_iter = 100000 * 2 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [30]:
import impl.LSTM as lstm
import impl.GRU as gru
import impl.RNN as rnn

# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [31]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3695
 tTcam rr J pm6d serirolhs earisc indarey bi3 aan the c'ptmd afetot 1 es Nite abd Warn nLpz atd iant


Iter-2000 loss: 2.7245
ou. Sh Ceteeso ea sfl, arce hurey bouperil Jamepdsundg vonte ir masintiso and io taclof th if erpas 


Iter-3000 loss: 2.3022
rire. Thy trec5and in ariof is the elpertly ing af if las pfirs liNopi8-s y7 e Rhis of im the Cacbou


Iter-4000 loss: 2.0026
ulkDy Mthe fourUcy 2–2iB't op ofBthe Emd texea, the P2ond i8 Aan the mourst wonta Sty livint io. is 


Iter-5000 loss: 1.7738
condtin ie, ftithe arddet dinstaldst anes aigal 47perye 209KeL, Japan8, Japan was of in moillai anat


Iter-6000 loss: 1.5963
ourt-Torin tevsea oftin tar iscecomithi nmercounc a. Japan is las of and io the cereg ta cesturuc, w


Iter-7000 loss: 1.4340
atery NhoJapany, NHWkedebom "y the ouHthy Kon"As areterictat on mand of Sromlitaliwoth  enthe Tomy w


Iter-8000 loss: 1.3119
inth and perobe2Nicbnd Fivtal isstror, Japan hosteritativel maint . who ar milithins are tero J

Iter-25000 loss: 0.5754
rn mere toling the seara, Kored tho Sum ora. From the Seasu Nipent aro ato dh an toI and War eagith 


Iter-26000 loss: 0.5621
 usen the 193h mestary of the Seana in malitary by 186, the Empire wured leviled to "Kua and coptoth


Iter-27000 loss: 0.5244
hiss ricon2 Seanal cant I prou, itlangs areancilidingr, withe coud the Seacker, dankecef, th  wered 


Iter-28000 loss: 0.5076
vilg toky. Frof ats wixity. The or mententaid dict the nagingifings of whily and Risrof CDhing forg 


Iter-29000 loss: 0.5071
ighest rifens-d of.5% of Japan's erea an Glpeonst olak oropexct ind 1ts canktalaresea Japanmes wised


Iter-30000 loss: 0.4965
f the Seand and Human Peand inst the wired the Homaly in pourth tusth larosd ineatiof lorledes nthor


Iter-31000 loss: 0.4795
aintsory gict ifob the tijn aron an thin 19.
. The caurtion uctingkso part livont conktiledo Ty the 


Iter-32000 loss: 0.4524
kukural toys ald a Unth camituriensevex a dansed cintury wing ghian ar and easisa-as en

Iter-49000 loss: 0.3184
 in Asia to host fouttal ty whons. Aroutan cenalioy ly. From lith Sea, of cintovninsed and Human Dev


Iter-50000 loss: 0.3053
stitetel treete the Upper is elored in Chine wite the was GDP awethic propen in thelacand caunty of 


Iter-51000 loss: 0.3031
et ereang stretovened IIs u prol cined Japan's has ea, ofrth the porth langest exparcensid of the 12


Iter-52000 loss: 0.2803
ty of Japanal GDPan country in the Global Peace Index. Japan was the firs the world's teo foid popfl


Iter-53000 loss: 0.2782
anese milatary of Japan, the Eame ripenth canked forta ly 9w1rthe four larsed, Japan has maito forla


Iter-54000 loss: 0.2900
md wiry, folloyiollitinane IIsiona. The peper. The country is divided into a ment Indecound cauntry 


Iter-55000 loss: 0.2666
wo decades a pixter forlate Weationfrum the 12th century unticinakes fromed of the C2uth-lares of th


Iter-56000 loss: 0.2944
e "Land of lunes eporo 2umelest expar parcendion as a sivecoped country is Japtec names

Iter-73000 loss: 0.2206
anji that Itstrin stobilanoved of the Rising Sun". Japan is a strthis of and the National Diet preat


Iter-74000 loss: 0.2224
s. Japan is a member of the UN, the G7, the G8, and the G2pine oI tncenarge the 12th and darly with 


Iter-75000 loss: 0.2086
to open to nomechas rea who roled the atomy conet loveardy. Archaed of Japan ou, which maintris ar. 


Iter-76000 loss: 0.2160
 The fires collomica the UN, uthe popul resmea lymshith ind in the G7, the G8, and if lheg of tobean


Iter-77000 loss: 0.2263
ericina, Indind. The fouth the for-lamensito ih tho sourth largest city peater Tokyo and several sy 


Iter-78000 loss: 0.2175
an Development Index a deint the deviconally renopvenced into poji tive exgh shard Horge the tyoree 


Iter-79000 loss: 0.1849
ond Sino-Japanese Stoted in the world's third-largest economy. Archaeologiacterized Japan is in the 


Iter-80000 loss: 0.2122
he Rish it ating tumend fourth largest. Jfpiess Repions Tok on eled fowth largest Tokyo

Iter-97000 loss: 0.1810
ke up 98.5% of Japan was endeded to an and early 20th centuries. The ComSt ex6y bo tar and In expand


Iter-98000 loss: 0.1780
he Globakey was end in 1945 foummis a divine symbol of the nation, forch wird the atomilgins was rev


Iter-99000 loss: 0.1627
 of the Country in is oftin the OEuctar Oyy in the world. The Palled the "Lane eambp Empire from the


Iter-100000 loss: 0.1770
the Country in as a diviled Thecand efendivin eref filitho porte wor desixte period of lorlaging bom


Iter-101000 loss: 0.1674
ple lintery ronred, is trin 1868, which make up about ning pref consided in the Country Brand Index,


Iter-102000 loss: 0.1585
h the Summer and Winter Olympic Games.
. Japan is ranked first in the Country Brand Infins-danity of


Iter-103000 loss: 0.1804
r th revelomerthy its e prom 20126 Ochits the Nive exts a onftunty hea, ChWese: 日本 Nippen, war and e


Iter-104000 loss: 0.1881
leente teand in the north-larly from Westernvilitary sungr and Sincenadix. nate we

Iter-121000 loss: 0.1647
lomeration economy. Alchaen, ithing porDed a sticaca, largest economy by Nilion in the Global county


Iter-122000 loss: 0.1827
 the East China Sea of Paleo-flation-d its e piople mak oumpire pore dinghis an ded by Chine fiuded 


Iter-123000 loss: 0.1562
udget, used for self-lemens-kan to enor Japan was the first counte peanes I and Homeituren arenrt. J


Iter-124000 loss: 0.1504
 Emperor. Japanded into and of World of Japan, which is the 1868, Hidging porDP a, the G8, and it ic


Iter-125000 loss: 0.1742
nhth the world's eabith over 35 million residents and the world's third-larghis a lamin", anded rist


Iter-126000 loss: 0.1523
pan is in Chines. The country in the Global Peace". penterde, in the Global Peace Nepeoppecopere of 


Iter-127000 loss: 0.1577
ped country relcend in a9, anhive ourth ter Fbo thimg mtith in the Global Competicalare with the Ris


Iter-128000 loss: 0.1318
 expenally renounced in the sive to the largest aijo foxth of Japan is in Chind

Iter-145000 loss: 0.1414
ccessictolcainted firjo stand its resictor as a developeace 1937 earle to roport claelyof. The Great


Iter-146000 loss: 0.1170
nored of the Glo52 the East China Sea, China, Koren, wan end its empirglation orlecthe poprth the Se


Iter-147000 loss: 0.1861
–2016 and is conomy city of Japan is the world's tenth-largest exportercend insto part China, follom


Iter-148000 loss: 0.1651
lature called the National Diet. Japan is in 1947, Japan has maintained a devel poppear conflidiclin


Iter-149000 loss: 0.1389
 Japan en, which incleading global city in the world. The Greater Tobeateriesid of ists in Eusthe si


Iter-150000 loss: 0.1413
's land area. The G8,853 941 mal toxthe aomal the world's third-dard of lirghts with the Sea, China,


Iter-151000 loss: 0.1634
x. Japan to expand in the G7ith. The country in Asia to host both osu hingl of Hiroshima and Nagt of


Iter-152000 loss: 0.1322
the G8, and the G20 and is constronmed in the number of Nobel laureates of any 

Iter-169000 loss: 0.1348
 the UNSper pored. Tokyo, the cogits a inokuiand Sino-Japanes Panes II perchacing the forly with car


Iter-170000 loss: 0.1507
illion residents and peocate wort is from the Sea lort Asia. Located in the Pacific Taijicanas Takui


Iter-171000 loss: 0.1593
strexcand into 47 pref. The firstly hint of Japanese in the name of the Sea of Japan, the East China


Iter-172000 loss: 0.1129
ilitary cha ur a deved in exth Resired iofonst umprifti-largest exporter and Russia, 2hich callowis 


Iter-173000 loss: 0.1444
ies, it the world's fourth lead camy the 1868, Japan has mas end in or to the east China Sea ate was


Iter-174000 loss: 0.1529
Archaeological ror and percal uerballoghoss I pen". Japan is a stratovolcanic archipelago ol aides p


Iter-175000 loss: 0.1182
and Taiwan tofoT oundand intilaliving and Human Developmest culrct andag preshurd with. The kound by


Iter-176000 loss: 0.1481
-deven is a seprelorld Worud larchaeic peint in suver and Winter Olympic and Ru

Iter-193000 loss: 0.1228
evised constitution is of livenst iep of Japan's total population. In the nates areate udonth Japan,


Iter-194000 loss: 0.1348
the 12th ced its riftion, Japan was rustored as hearly an into 47, Japan is a developed country in E


Iter-195000 loss: 0.1376
res, is the highest-ranked alional mor d stored's an eloxts divines in the early 17th ofein is an Em


Iter-196000 loss: 0.1434
ed first ind the world's an ounthing pricteruman Duven period of increasing militarism. The Second S


Iter-197000 loss: 0.1243
onal monarchy 20th censiorla levidents and was proc peony. Archaeological rester Around. Aropided To


Iter-198000 loss: 0.1148
tary burgens Tat tolation, par Gfed's largest exporter and fourth-largest exporter and fourth lenid 


Iter-199000 loss: 0.1386
th largest the Cound Nippon-koku or Nihon-koku, "State of Japan") is an iso-Japan to and its eas of 


Iter-200000 loss: 0.1175
hest-ranked Asian country in the Global Competitivenese period of increas of Ja

In [ ]:
=========================================================================
Iter-297000 loss: 0.1208
=========================================================================
ry with a Unised Japan's history. From the 12th century, which isHomAreat Divi al China, folicter Ol
=========================================================================


=========================================================================
Iter-298000 loss: 0.0927
=========================================================================
 the number of Nobel laureates of anI arean Index. Japan was the first cores. Japan is a developed c
=========================================================================


=========================================================================
Iter-299000 loss: 0.1143
=========================================================================
ake up Jcaly Id in 1868 and the Empire of Japan was proclaimed, ater popJdallky, thicalic and eanly 
=========================================================================


=========================================================================
Iter-300000 loss: 0.1084
=========================================================================
uled by suchaed into a and in 1945 follodud of Hurigh in the world. The Greater Tokyo Area, which in
=========================================================================


Out[27]:
<__main__.GRU at 0x10d06ed68>

In [ ]:

=========================================================================
Iter-198000 loss: 0.1331
=========================================================================
tary shoguns who ruled in the nainty n mainterd Taiwat lors latulaty und Sunarend thia dudg oonaly r
=========================================================================


=========================================================================
Iter-199000 loss: 0.1386
=========================================================================
th largest imf in the korlation mainlatI un the eipan endand of the Rising Sun". Japan is a stratovo
=========================================================================


=========================================================================
Iter-200000 loss: 0.0987
=========================================================================
hest-ranked Asian countrord with the Since caly towertor Jaty in the country in East Asia. Located i
=========================================================================


Out[11]:
<__main__.GRU at 0x10d053f98>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1890
=========================================================================
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 
=========================================================================


=========================================================================
Iter-100000 loss: 0.1673
=========================================================================
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch
=========================================================================


Out[24]:
<__main__.GRU at 0x10d1f7b00>

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>